In [ ]:
# Imports from standard library
import os.path
import datetime
import warnings
warnings.filterwarnings('ignore')

# Import of third party packages
import numpy as np
import netCDF4
%matplotlib widget
from matplotlib import pyplot
from matplotlib import dates
import cmocean

# Configuration
setup_dir = "."           # directory with GOTM model setup
ensemble_dir = "."        # directory with GOTM model setup
varname = "temp"          # name of model variable to plot
N = 20                    # ensemble size
max_temp_diff = 2.0       # maximum difference (°C) to show in contour plot

In [ ]:
# Read satellite observations of sea surface temperature
with open(os.path.join(setup_dir, 'cci_sst.dat')) as f:
    obs = [line.rstrip().split('\t') for line in f if not line.startswith('#')]
obs_times = dates.date2num([datetime.datetime.strptime(o[0], '%Y-%m-%d %H:%M:%S') for o in obs])
obs_values = np.array([o[1] for o in obs], dtype=float)

## Plot default GOTM results

Forecast-only, no data assimilation

In [ ]:
nc = netCDF4.Dataset(os.path.join(setup_dir, 'result.nc'))

# Read model temperature and coordinates
time = netCDF4.num2date(nc['time'], nc['time'].units)
mpltime = dates.date2num(time)
z = nc['z'][:, :, 0, 0]
ncvar = nc[varname]
temp = ncvar[:, :, 0, 0]
sst = temp[:, -1]

fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(figsize=(8,8), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex='col')

# Plot modelled and observed sea surface temperature
ax1.plot_date(obs_times, obs_values, '.k', alpha=0.4, label='satellite')
ax1.plot_date(mpltime, sst, '-', label='model')
ax1.set_xlim(mpltime[0], mpltime[-1])
ax1.set_ylabel('temperature (°C)')
ax1.grid()
ax1.legend()
ax1.set_title('sea surface temperature')
cax1.axis('off')

# Plot modelled temperature throughout the water column
mpltime_2d = np.broadcast_to(mpltime[:, np.newaxis], z.shape)
pc = ax2.contourf(mpltime_2d, z, temp, cmap=cmocean.cm.thermal)
cb = fig.colorbar(pc, cax=cax2)
cb.set_label('temperature (°C)')
ax2.set_ylabel('height (m)')
ax2.set_title('model temperature')
ax2.grid()
ax2.xaxis.axis_date()

## Plot ensemble GOTM results

In [ ]:
# Open NetCDF output of all ensemble members
ncs = [netCDF4.Dataset(os.path.join(ensemble_dir, 'result_%04i.nc' % (i + 1))) for i in range(N)]

# Read model temperature and coordinates
enstime = netCDF4.num2date(ncs[0]['time'], ncs[0]['time'].units)
ensmpltime = dates.date2num(enstime)
z = ncs[0].variables['z'][:, :, 0, 0]
temps = [nc[varname][:, :, 0, 0] for nc in ncs]
ssts = [temp[:, -1] for temp in temps]

# Plot ensemble mean sea surface temperature, along with original (no DA) result and observations
fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(figsize=(8,8), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex='col')
ax1.plot_date(obs_times, obs_values, '.k', alpha=0.4, label='satellite')
ax1.plot_date(time, sst, '-', label='model, no DA')
ax1.plot_date(enstime, np.mean(ssts, axis=0), '-', label='model, DA')
#ax1.set_xlim(time[0], time[-1])
ax1.set_xlim(datetime.datetime(2022,1,1), time[-1])
ax1.set_ylabel('temperature (°C)')
ax1.grid()
ax1.legend()
ax1.set_title('sea surface temperature')
cax1.axis('off')

# Plot ensemble mean temperature throughout the water column
mpltime_2d = np.broadcast_to(ensmpltime[:, np.newaxis], z.shape)
pc = ax2.contourf(mpltime_2d, z,  np.mean(temps, axis=0))
cb = fig.colorbar(pc, cax=cax2)
cb.set_label('temperature (°C)')
ax2.set_ylabel('height (m)')
ax2.set_title('temperature');
ax2.grid()
ax2.xaxis.axis_date()

# Impact of data assimilation on temperature throughout the water column

In [ ]:
# Plot ensemble mean sea surface temperature, along with original (no DA) result and observations
fig, ((ax1, cax1), (ax2, cax2), (ax3, cax3)) = pyplot.subplots(figsize=(8,8), nrows=3, ncols=2, width_ratios=[0.95, 0.05], sharex='col')
ax1.plot_date(obs_times, obs_values, '.k', alpha=0.4, label='satellite')
ax1.plot_date(time, sst, '-', label='model, no DA')
ax1.plot_date(enstime, np.mean(ssts, axis=0), '-', label='model, DA')
#ax1.set_xlim(time[0], time[-1])
ax1.set_xlim(datetime.datetime(2021,1,1), time[-1])
ax1.set_ylabel('temperature (°C)')
ax1.grid()
ax1.legend()
ax1.set_title('sea surface temperature')
cax1.axis('off')

temp_diff = np.mean(temps, axis=0) - temp[-enstime.size:, :]
contours = np.linspace(-max_temp_diff, max_temp_diff, 21)
pc = ax2.contourf(mpltime_2d, z, temp, cmap=cmocean.cm.thermal)
cb = fig.colorbar(pc, cax=cax2)
cb.set_label('temperature (°C)')
ax2.set_ylabel('height (m)')
ax2.set_title('temperature (no DA)');
ax2.grid()
ax2.xaxis.axis_date()

temp_diff = np.mean(temps, axis=0) - temp[-enstime.size:, :]
contours = np.linspace(-max_temp_diff, max_temp_diff, 21)
pc = ax3.contourf(mpltime_2d, z, temp_diff, cmap='RdBu_r', levels=contours, extend='both')
cb = fig.colorbar(pc, cax=cax3)
cb.set_label('temperature difference (°C)')
ax3.set_ylabel('height (m)')
ax3.set_title('temperature difference (DA - no DA)');
ax3.grid()
ax3.xaxis.axis_date()


In [ ]:
fig, ax = pyplot.subplots(figsize=(8, 8))
temp_diff = np.mean(temps, axis=0) - temp[-enstime.size:, :]
contours = np.linspace(-max_temp_diff, max_temp_diff, 21)
pc = ax.contourf(mpltime_2d, z, temp_diff, cmap='RdBu_r', levels=contours, extend='both')
cb = fig.colorbar(pc, ax=ax)
cb.set_label('temperature difference (°C)')
ax.set_ylabel('height (m)')
ax.set_title('temperature difference (DA - no DA)');
ax.grid()
ax.xaxis.axis_date()
ax.set_xlim(datetime.datetime(2022,1,1), time[-1])